In [1]:

# conda install pytorch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 -c pytorch -y & conda install pyg -c pyg -y & conda install -c anaconda networkx -y & conda install -c conda-forge matplotlib -y

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch_geometric.data import Data
import networkx as nx
import matplotlib.pyplot as plt
import torch_geometric.utils as utils
from utils import *
from model import UpdateRule


height = 1
width = 3
hidden_dim = 8

n_inputs = 2
n_outputs = 3


update_rule = UpdateRule(
    n_inputs, 
    n_outputs,
    hidden_dim,
    4
)
update_rule.build_graph(
    height,
    width
)


# cuda_device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
cuda_device = torch.device("cpu")

update_rule = update_rule.to(cuda_device)

update_rule.draw()

NameError: name 'update_rule' is not defined

In [ ]:
from datasets import *
import copy
import numpy as np
np.set_printoptions(precision=1, suppress=True)


# dataset_loader = DataLoader(TranslateDataset(), batch_size=1, shuffle=True)

meta_set = MetaDataset()

optimizer = torch.optim.Adam(update_rule.parameters(), lr=0.001)

best_loss = 10000
best_model = None

n_steps = 2
batch_size = 5
for epoch in range(10000):
    loss = 0
    for _ in range(batch_size):
        for set_idx in meta_set.iterate():
            update_rule.reset() 
            x = update_rule.initial_state(height, width)
            # for idx, (problem_data_x, problem_data_y) in enumerate(meta_set.get_set(set_idx)):
                
        
            x, batch_loss, network_output, correct = update_rule(x, n_steps, meta_set.get_set(set_idx))
            loss += batch_loss
                
            # network_output = update_rule.get_output(x)
            # loss = F.mse_loss(problem_data_y.float().squeeze(0), network_output)
            # loss += F.mse_loss(problem_data_y.float().squeeze(0), network_output)
            
            #cross entropy loss
            # loss += F.binary_cross_entropy_with_logits(network_output, problem_data_y.float().squeeze(0))
           
            
    if loss < best_loss:
        # best_model = copy.deepcopy(update_rule)
        best_loss = loss
     
    loss /= batch_size   
    loss.backward()
    
    nn.utils.clip_grad_norm_(update_rule.parameters(), 1)
    optimizer.step()  
    optimizer.zero_grad()  
        
    print(f"""\r 
          Epoch {epoch * batch_size} |
          Loss {loss:.6} |
          Network out: {network_output} |
          Correct:  {correct}
        """.replace("\n", " ").replace("          ", ""), end="")
    
    print(f"\r Epoch {epoch * batch_size} | Loss {loss}", end="")
    if epoch % (200 // batch_size) == 0:
        print()


 Epoch 0 | Loss 2.0328893661499023 out: [0. 1. 0.] | Correct:  [0. 1. 0.]         
 Epoch 200 | Loss 1.6995563507080078 out: [0. 0. 1.] | Correct:  [0. 0. 1.]         
 Epoch 300 | Loss 1.6995595693588257 out: [0. 1. 0.] | Correct:  [0. 1. 0.]         

KeyboardInterrupt: 

In [ ]:
import numpy as np
np.set_printoptions(precision=3)

run_rule(torch.tensor([[0,0,0,1]]), update_rule)

TypeError: forward() missing 1 required positional argument: 'data'

In [ ]:

def run_rule_meta(update_rule, training, test, n_steps):
    x = update_rule.initial_state(height, width)
    for (problem_x, problem_y) in training:
        print(problem_x)
        for _ in range(n_steps):
            x = update_rule(x, problem_x.float(), problem_y.float())
            
            
    for _ in range(n_steps):
        x = update_rule(x, test.float())
        
    network_output = update_rule.get_output(x).detach()
    return network_output
    

In [ ]:
# run_rule_meta(update_rule, torch.tensor([
#     ([1,0,0,0], [0,1,0,0]),
#     ([0,1,0,0], [0,0,1,0]),
#     ([0,0,1,0], [0,0,0,1]),
#     ]), torch.tensor([0,1,0,0]), 8)

# run_rule_meta(best_model, torch.tensor([
#     ([1,0,0,0,0,0,0], [0,1,0,0,0,0,0]),
#     ([0,1,0,0,0,0,0], [0,0,1,0,0,0,0]),
#     # ([0,0,1,0,0,0,0], [0,1,0,0,0,0,0]),
#     ([0,0,0,1,0,0,0], [0,0,0,0,1,0,0]),
#     ([0,0,0,0,1,0,0], [0,0,0,0,0,1,0]),
#     ([0,0,0,0,0,1,0], [0,0,0,0,0,0,1]),
#     ([0,0,0,0,0,0,1], [1,0,0,0,0,0,0]),
#     ]), torch.tensor([0,0,1,0,0,0,0]), n_steps)

run_rule_meta(best_model, torch.tensor([
    ([1,0,0,0,0,0,0], [0,0,0,0,0,0,1]),
    ([0,1,0,0,0,0,0], [1,0,0,0,0,0,0]),
    # ([0,0,1,0,0,0,0], [0,1,0,0,0,0,0]),
    ([0,0,0,1,0,0,0], [0,0,1,0,0,0,0]),
    ([0,0,0,0,1,0,0], [0,0,0,1,0,0,0]),
    ([0,0,0,0,0,1,0], [0,0,0,0,1,0,0]),
    ([0,0,0,0,0,0,1], [0,0,0,0,0,1,0]),
    ]), torch.tensor([0,0,1,0,0,0,0]), n_steps)

In [ ]:
update_rule.initial

In [ ]:
test = torch.rand([1,5])
#repeat on first dimension 3 times without using .repeat
test = test.expand(3, -1)
test

In [ ]:
meta_set = MetaDataset()

In [ ]:
print(torch.stack(list(meta_set.get_set(0))[0], 1))
print(torch.stack(list(meta_set.get_set(0))[1], 1))


In [ ]:
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

loader = METRLADatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=5)

from torch_geometric_temporal.signal import temporal_signal_split
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

for snapshot in train_dataset:
    print(snapshot.x.shape)
    print(snapshot.y.shape)
    break